In [4]:
import pandas as pd
import numpy as np
import json

import plotly.io as pio
from plotly import graph_objects
import plotly.express as px
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.graph_objects as go

In [5]:

data = pd.read_csv('economic_freedom_index2019_data.csv', encoding='latin-1')
null_c = data[data['World Rank'].isnull()] ## countries w nan values, need to be removed
data = data[data['World Rank'].notna()] #### remove those countries
no = data.columns[data.isna().any()].tolist() ### columns w nan values, need to be removed
data = data.drop(no,axis=1) #### remove those columns
data = data.drop(['Country'],axis=1) ### Drop column b/c it messes w loop

columns = list(data.columns.values) 

# ##############################################################

data = data.replace(',','', regex=True)
data[columns[22]] = data[columns[22]].str.replace('$', '')
data[columns[24]] = data[columns[24]].str.replace('$', '')
data.loc[88,[columns[22]]] = 40 ###hardcode
data.loc[88,[columns[24]]] = 1700 ##hardcode

dataECON = data
dataHD = pd.read_csv('human_development.csv', encoding='latin-1')
dataHD = dataHD.iloc[:188,[i for i in range(8) ] ]
null_data = dataHD[dataHD.isnull().any(axis=1)]
dataEDU = dataHD.loc[:,['Country','Mean Years of Education']]
#dataEDU = dataHD.iloc[:188,[i for i in range(8) ] ]
dataEDU=  dataEDU.set_index(['Country'])
dataECON = dataECON.set_index(['Country Name'])
dataECON = dataECON['World Rank']
df = pd.merge(dataECON,dataEDU,left_index=True, right_index=True)


In [6]:

traces=[]
metrics = np.array(['World Rank','Mean Years of Education'])
booly = [True,False]

for metric,i in zip(metrics , range(len(metrics))):
    traces.append(go.Choropleth(locations=df.index.tolist(),
                                locationmode='country names',
                                z=df[metric],
                                colorscale='Portland',
                                marker_line_color='darkgray',
                                marker_line_width=0.5,
                                text= df.index.tolist(),
                                reversescale=booly[i],
                                colorbar = {'title':metric, 'len':500,'lenmode':'pixels' },
                                visible=True if metric== 'World Rank' else False) )
    
updatemenus = []
metrics = np.array(['World Rank','Mean Years of Education'])

buttons=[]
for metric in metrics:
    buttons.append(dict(method='update',
                        label=metric,
                        args=[{'visible': metrics==metric}]))
    
dropdown = dict(buttons=buttons, direction='down',x = 0.01,xanchor = 'left',
                y = 0.99,yanchor = 'bottom',font = dict(size=11))
updatemenus=[dropdown] 
layout = dict(updatemenus=updatemenus,
              title='World Rank vs. Education')
    
col_map = go.Figure(data = traces,layout = layout)
iplot(col_map)